<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/LogisticRegression/LR_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classification by  LogisticRegression and CountVectorizer**


------------------



 1- إستيراد المكتبات

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import nltk
nltk.download('stopwords')
from nltk.stem.isri import ISRIStemmer
import re
import string
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


--------

2- استيراد ملفات البيانات

In [2]:
# df = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/3.xlsx")
df = pd.read_excel("drive/MyDrive/mining/4.xlsx")
df.head()

,text,targe
0,أكد وزير الاتصال الناطق الرسمي باسم الحكومة مص...,0
1,عبدالاله بوسحابة أخبارنا المغربية أثار فيلم أل...,0
2,عبدالاله بوسحابة اخبارنا المغربية أبدى القائمو...,0
3,أخبارنا المغربية اختار الفنان المغربي البشير ع...,0
4,المادة الأولى على ملك غرناطة القادة الفقهاء ال...,0


------
3- shuffle  
بعثرة البيانات

In [3]:
df=shuffle(df)
df

,text,targe
55925,أعلن المكتب المركزي للأبحاث القضائية التابع لل...,1
8769,صرح الرئيس المدير العام لمجموعة سي إم سي جي إم...,2
27431,مراكش المغربية ولدى وصوله إلى موقع الحفل وجد ص...,3
49201,يبدو أن الرقم القياسي المسجل لأطول مباراة كرة ...,4
46434,نظمت سفارة المغرب بطوكيو الاثنين الماضي حفلا ك...,3
...,...,...
52861,يتفاعل مع الثورات العربية ويبث في شهر رمضان ال...,0
17455,تقدم فريق عمل فيلم محمد إمام نجل النجم عادل إم...,0
41716,أشرف الملك محمد السادس مرفوقا بالأمير مولاي اس...,2
61238,قال عبد الرحيم خيي بابا رئيس غرفة التجارة والص...,2


------
4- clean
تنظيف البيانات

In [4]:
print(np.sum(df.isnull().any(axis=1)))

clean_df= df.dropna()
print(np.sum(clean_df.isnull().any(axis=1)))
#-------------
print (df.notnull().any(axis = 0))
print(len(df))
#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
        
remove_hashtag(clean_df)
clean_df
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc

stemmer = ISRIStemmer()
clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
print(clean_df)


378
0
text     True
targe    True
dtype: bool
68690
                                                    text  targe
55925  اعلن المكتب المركزي لابحاث القضائيه التابع لمد...      1
8769   صرح الرئيس المدير العام لمجموعه سي ام سي جي ام...      2
27431  مراكش المغربيه ولدي وصوله الي موقع الحفل وجد ص...      3
49201  يبدو ان الرقم القياسي المسجل لاطول مباراه كره ...      4
46434  نظمت سفاره المغرب بطوكيو الاثنين الماضي حفلا ك...      3
...                                                  ...    ...
52861  يتفاعل مع الثورات العربيه ويبث في شهر رمضان ال...      0
17455  تقدم فريق عمل فيلم محمد امام نجل النجم عادل ام...      0
41716  اشرف الملك محمد السادس مرفوقا بالامير مولاي اس...      2
61238  قال عبد الرحيم خي بابا رئيس غرفه التجاره والصن...      2
47662  انعقد اليوم الاثنين بالرباط اجتماع تراسه رئيس ...      3

[68312 rows x 2 columns]


---------------
5- accuracy calculation

عمل تابع لحساب الدقة

In [5]:
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

------
6- classifier:LogisticRegression & TfidfVectorizer

with Cross validation

التنصنيف عبر خوارزمية

LogisticRegression

مع ميزة

TfidfVectorizer


استخدام خوارزمية
 
Cross validation

 لتوزيع عينات الاختبار والتأكد من عدم وجود

 Overfitting

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

x =clean_df['text'].values
y = clean_df['targe'].values

strtfdKFold = StratifiedKFold(n_splits=4)
kfold = strtfdKFold.split(x, y)

classifier = LogisticRegression()
tfidf = TfidfVectorizer()

result_LogisticRegressionClassifier=[]

for k, (train_index, test_index) in enumerate(kfold):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    tfidf.fit(x_train)
    X_train = tfidf.transform(x_train)
    X_test  = tfidf.transform(x_test)
    X_train
    model=classifier.fit(X_train, y_train)
    y_pred=(model.predict(X_test))
    result_LogisticRegressionClassifier.append(calculate_results(y_test, y_pred))

result_LogisticRegressionClassifier

[{'accuracy': 94.93500409884061,
  'f1': 0.9493445741717786,
  'precision': 0.9493807658586266,
  'recall': 0.9493500409884061},
 {'accuracy': 94.9818479915681,
  'f1': 0.9498174465265411,
  'precision': 0.949835005199652,
  'recall': 0.949818479915681},
 {'accuracy': 94.88816020611313,
  'f1': 0.9488705587606279,
  'precision': 0.948918515930805,
  'recall': 0.9488816020611313},
 {'accuracy': 94.80618339384003,
  'f1': 0.9480554575407977,
  'precision': 0.9480982957473005,
  'recall': 0.9480618339384003}]

In [7]:
accuracy=0
f1=0
precision=0
recall=0
for i in result_LogisticRegressionClassifier:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(result_LogisticRegressionClassifier)
f1=f1/len(result_LogisticRegressionClassifier)
precision=precision/len(result_LogisticRegressionClassifier)
recall=recall/len(result_LogisticRegressionClassifier)
# (result_DecisionTreeClassifier)
results_LogisticRegressionClassifier=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
results_LogisticRegressionClassifier

[{'accuracy': 94.90279892259048,
  'f1': 0.9490220092499363,
  'precision': 0.9490581456840961,
  'recall': 0.9490279892259046}]

----------------
------------------
-----------------
7-Results

النتائج

نتائج التدريب لكل قاعدة بيانات

In [9]:
print('LogisticRegressionClassifier results in dataset 1:')
results_LogisticRegressionClassifier

LogisticRegressionClassifier results in dataset 1:


[{'accuracy': 96.5497342348224,
  'f1': 0.9655524254330433,
  'precision': 0.9656931250701035,
  'recall': 0.965497342348224}]

In [8]:
print('LogisticRegressionClassifier results in dataset 2:')
results_LogisticRegressionClassifier

LogisticRegressionClassifier results in dataset 2:


[{'accuracy': 96.39764461378593,
  'f1': 0.9640159630584807,
  'precision': 0.96409310399888,
  'recall': 0.9639764461378594}]

In [8]:
print('LogisticRegressionClassifier results in dataset 3:')
results_LogisticRegressionClassifier

LogisticRegressionClassifier results in dataset 3:


[{'accuracy': 95.27937889066929,
  'f1': 0.9527979646709726,
  'precision': 0.952834102572899,
  'recall': 0.9527937889066931}]

In [8]:
print('LogisticRegressionClassifier results in dataset 4:')
results_LogisticRegressionClassifier

LogisticRegressionClassifier results in dataset 4:


[{'accuracy': 94.90279892259048,
  'f1': 0.9490220092499363,
  'precision': 0.9490581456840961,
  'recall': 0.9490279892259046}]

In [9]:
Final_Result=[
  {'accuracy': 96.35964339589191,
  'f1': 0.9636438504186007,
  'precision': 0.963822932097349,
  'recall': 0.9635964339589191}
  ,
  {'accuracy': 96.39764461378593,
  'f1': 0.9640159630584807,
  'precision': 0.96409310399888,
  'recall': 0.9639764461378594}
  ,  
  {'accuracy': 95.27937889066929,
  'f1': 0.9527979646709726,
  'precision': 0.952834102572899,
  'recall': 0.9527937889066931}
  ,
  {'accuracy': 94.90279892259048,
  'f1': 0.9490220092499363,
  'precision': 0.9490581456840961,
  'recall': 0.9490279892259046}
  ]

accuracy=0
f1=0
precision=0
recall=0
for i in Final_Result:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(Final_Result)
f1=f1/len(Final_Result)
precision=precision/len(Final_Result)
recall=recall/len(Final_Result)
# (result_DecisionTreeClassifier)
Final_Result=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]

print('Final_Result:')
(Final_Result)

Final_Result:


[{'accuracy': 95.7348664557344,
  'f1': 0.9573699468494976,
  'precision': 0.9574520710883061,
  'recall': 0.9573486645573441}]